In [2]:
import os

In [3]:
%pwd

'c:\\Users\\ASUS\\basic_python_ program\\iNeuron_end_full_project\\ML_Proj_default\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\ASUS\\basic_python_ program\\iNeuron_end_full_project\\ML_Proj_default'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    target_column: str

In [13]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target_column = schema.name
        )

        return data_transformation_config


In [15]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

#-----------------------------My changes
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [33]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up
    def scaling(self,train_x,test_x):
        # Standarizing with standard scaler
        scaler_x = MinMaxScaler()
        x_train = scaler_x.fit_transform(train_x)
        x_test= scaler_x.transform(test_x)
        
        return x_train,x_test

    def PCA(self,train_x,test_x):
        pca = PCA(random_state=10)
        pca.fit(train_x)
        # Let's take 10 , which explains 98.5 %
        pc_selected = PCA(n_components=10, random_state=10)
        # This is the new transformed data
        x_train_pc = pc_selected.fit_transform(train_x)
        x_test_pc = pc_selected.transform(test_x)
        
        return x_train_pc,x_test_pc


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        
       

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)
        print(type(train))
        
        
        train_x = train.drop([self.config.target_column], axis=1)
        test_x = test.drop([self.config.target_column], axis=1)
        train_y = train[[self.config.target_column]]
        test_y = test[[self.config.target_column]]
        print(type(train_y))
        
        train_x,test_x = self.scaling(train_x,test_x)
        train_x,test_x = self.PCA(train_x,test_x)
        
        train_x = pd.DataFrame(train_x)
        test_x = pd.DataFrame(test_x)
        
        train = train_x
        train[self.config.target_column] = train_y
        
        test = test_x
        test[self.config.target_column] = test_y
        
        print(train)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)
    

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
    
   
        


In [34]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-05-21 20:59:18,817: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-21 20:59:18,818: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-21 20:59:18,821: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-21 20:59:18,822: INFO: common: created directory at: artifacts]
[2024-05-21 20:59:18,823: INFO: common: created directory at: artifacts/data_transformation]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
             0         1         2         3         4         5         6  \
0    -0.329183  0.078859 -0.203239 -0.082555  0.170357 -0.100812 -0.092476   
1     0.435290 -0.103901 -0.172480  0.234492 -0.108572  0.095934 -0.044130   
2     0.256033 -0.118699 -0.068036 -0.123261  0.002880 -0.031499 -0.060958   
3    -0.252369  0.125362 -0.136815  0.026815  0.052542 -0.055458 -0.093892   
4     0.005348  0.329422  0.088771  0.170929  0.025429  0.019128 -0.213768   
...        ...       ...       